In [1]:
import pickle
from collections import namedtuple
import numpy as np
import cv2
import os
import matplotlib.image as mpimg
import h5py
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
from glob import glob

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
TRAIN_DIR_IMGS = '../data/Train/'
TRAIN_DIR_DOTTED_IMGS = '../data/TrainDotted/'
NUM_TRAIN_IMGS = 948
NUM_VAL_IMGS = 70

PATCH_SIZE = 256
PAD_SIZE = 16
NEG_RATIO = 0.02

WITH_PUPS = True

TEST_DIR_IMGS = '../data/Test/'

In [3]:
SeaLionCoord = namedtuple('SeaLionCoord', ['img_id', 'x', 'y', 'cls'])

In [4]:
COLORS = (  (243, 8, 5),          # red
            (244, 8, 242),        # magenta
            (87, 46, 10),         # brown
            (25, 56, 176),        # blue
            (38, 174, 21),        # green
)

In [5]:
BAD_IMGS = [3, 7, 9, 21, 30, 34, 71, 81, 89, 97, 151, 184, 215, 234,
            242, 268, 290, 311, 331, 344, 380, 384, 406, 421, 469, 475, 490,
            499, 507, 530, 531, 605, 607, 614, 621, 638, 644, 687, 712, 721,
            767, 779, 781, 794, 800, 811, 839, 840, 869, 882, 901, 903, 905,
            909, 913, 927, 946, 912, 66, 235, 292, 426, 529, 593, 643, 816, 857]

In [6]:
ID_IMGS = []
for i in range(NUM_TRAIN_IMGS):
    if i not in BAD_IMGS:
        ID_IMGS.append(i)
ID_IMGS = shuffle(ID_IMGS)
print(len(ID_IMGS))

881


In [7]:
ID_TRAIN_IMGS = ID_IMGS[NUM_VAL_IMGS:]
ID_VAL_IMGS = ID_IMGS[:NUM_VAL_IMGS]

In [8]:
train_h5py = h5py.File("data/train_all_classes.h5",mode='w')
dset_target_imgs = train_h5py.create_dataset("target_imgs", (0, PATCH_SIZE + 2*PAD_SIZE, PATCH_SIZE + 2*PAD_SIZE, 1),
                                             maxshape=(None, PATCH_SIZE + 2*PAD_SIZE, PATCH_SIZE + 2*PAD_SIZE, 1),
                                             dtype=np.uint8, shuffle=True, compression="gzip")

dset_imgs = train_h5py.create_dataset("imgs", (0, PATCH_SIZE, PATCH_SIZE, 3),
                                      maxshape=(None, PATCH_SIZE, PATCH_SIZE, 3),
                                      dtype=np.uint8, shuffle=True, compression="gzip")

dset_counts = train_h5py.create_dataset("counts", (0,), maxshape=(None,),
                                        dtype=np.uint8, shuffle=True, compression="gzip")

In [8]:
def neighbourhood(xs, ys):
    return sum((x-y)**2 for x, y in zip(xs, ys)) > 300
        
def remove_neighbourhood(data, cond):
    coords = []
    for element in data:
        if all(cond(element, other) for other in coords):
            coords.append(element)
    return coords

In [9]:
def remove_some_negative(x, y, c, negative_ratio=1.0):
    win_mask = c != 0
    los_mask = c == 0

    win_x = x[win_mask]
    win_y = y[win_mask]
    win_c = c[win_mask]
    los_x = x[los_mask]
    los_y = y[los_mask]
    los_c = c[los_mask]

    order = np.random.permutation(los_c.shape[0])
    amt = int(win_c.shape[0]*negative_ratio)
    los_x = los_x[order][0:amt]
    los_y = los_y[order][0:amt]
    los_c = los_c[order][0:amt]

    both_x = np.concatenate([win_x, los_x])
    both_y = np.concatenate([win_y, los_y])
    both_c = np.concatenate([win_c, los_c])
    order = np.random.permutation(both_c.shape[0])

    return both_x[order], both_y[order], both_c[order]

In [10]:
def create_datasets(LIST_IDS, DSET_target_imgs, DSET_imgs, DSET_counts): 
    for ID in LIST_IDS: 
        print("\tProcessing sample dset_target_imgs, with id={}".format(ID))
        img = mpimg.imread(os.path.join(TRAIN_DIR_IMGS,"{}.jpg".format(ID)))
        dotted_img = mpimg.imread(os.path.join(TRAIN_DIR_DOTTED_IMGS, "{}.jpg".format(ID)))

        # extract coordinates of animals : 
        diff = cv2.subtract(img, dotted_img)
        diff[dotted_img < 30] = 0
        x, y = np.where(np.any(diff > 60, axis=2))
        xy = zip(x, y)
        coords = remove_neighbourhood(xy, neighbourhood)
        sealioncoords = [ SeaLionCoord(100, x, y, np.argmin(np.linalg.norm(COLORS - dotted_img[x, y], axis=1)))
                         for x, y in coords]

        # make regular crops of size PATHCH_SIZE
        n_x = img.shape[0] // PATCH_SIZE
        n_y = img.shape[1] // PATCH_SIZE
        x_rem = img.shape[0] % PATCH_SIZE
        y_rem = img.shape[1] % PATCH_SIZE

        imgs = []
        target_imgs = []
        counts = []

        for x in range(n_x + (x_rem > 0)):
            for y in range(n_y + (y_rem > 0)):
                count = 0
                xmin = x*PATCH_SIZE
                ymin = y*PATCH_SIZE
                xmax = xmin+PATCH_SIZE
                ymax = ymin+PATCH_SIZE

                new_img = img[xmin:xmax, ymin:ymax]

                # Edge case
                if x == n_x or y == n_y:
                    temp = np.zeros((PATCH_SIZE, PATCH_SIZE, 3), dtype=np.uint8)
                    temp[:new_img.shape[0], :new_img.shape[1], :] = new_img
                    new_img = temp

                target_img = np.zeros((PATCH_SIZE + 2*PAD_SIZE, PATCH_SIZE + 2*PAD_SIZE), dtype=np.uint8)

                for i in sealioncoords:
                    if i.cls == 4 and not WITH_PUPS: # Skip pups
                        continue

                    # In 256x256-coords
                    unpad_x = i.x - xmin
                    unpad_y = i.y - ymin
                    if unpad_x < 0 or unpad_y < 0 or unpad_x >= PATCH_SIZE or unpad_y >= PATCH_SIZE:
                        continue

                    # In 288x288-coords
                    pad_x = unpad_x + PAD_SIZE
                    pad_y = unpad_y + PAD_SIZE
                    assert pad_x - PAD_SIZE >= 0
                    assert pad_x + PAD_SIZE < PATCH_SIZE + 2*PAD_SIZE
                    assert pad_y - PAD_SIZE >= 0
                    assert pad_y + PAD_SIZE < PATCH_SIZE + 2*PAD_SIZE

                    target_img[(pad_x - PAD_SIZE):(pad_x + PAD_SIZE), (pad_y - PAD_SIZE):(pad_y + PAD_SIZE)] += 1
                    count += 1

                imgs.append(new_img)
                target_imgs.append(target_img)
                counts.append(count)

        # convert imgs, target_imgs and counts into numpy arrays :
        imgs = np.asarray(imgs, dtype=np.uint8)
        target_imgs = np.asarray(target_imgs, dtype=np.uint8)
        counts = np.asarray(counts, dtype=np.uint8)
        
        # keep only 5% of blanck patches ! 
        imgs, target_imgs, counts = remove_some_negative(imgs, target_imgs, counts, negative_ratio=NEG_RATIO)
        num_new_samples = imgs.shape[0]
        if num_new_samples > 0 :
            # add these elements to the datasets : 
            DSET_target_imgs.resize(DSET_target_imgs.shape[0] + num_new_samples, axis=0)
            DSET_target_imgs[-num_new_samples:] = np.expand_dims(target_imgs, axis=-1)

            DSET_imgs.resize(DSET_imgs.shape[0] + num_new_samples, axis=0)
            DSET_imgs[-num_new_samples:] = imgs

            DSET_counts.resize(DSET_counts.shape[0] + num_new_samples, axis=0)
            DSET_counts[-num_new_samples:] = counts

In [12]:
%%time

create_datasets(ID_TRAIN_IMGS, dset_target_imgs, dset_imgs, dset_counts)
print(dset_imgs.shape)
print(dset_target_imgs.shape)
print(dset_counts.shape)

	Processing sample dset_target_imgs, with id=745
	Processing sample dset_target_imgs, with id=815
	Processing sample dset_target_imgs, with id=205
	Processing sample dset_target_imgs, with id=288
	Processing sample dset_target_imgs, with id=437
	Processing sample dset_target_imgs, with id=863
	Processing sample dset_target_imgs, with id=82
	Processing sample dset_target_imgs, with id=64
	Processing sample dset_target_imgs, with id=484
	Processing sample dset_target_imgs, with id=683
	Processing sample dset_target_imgs, with id=346
	Processing sample dset_target_imgs, with id=390
	Processing sample dset_target_imgs, with id=491
	Processing sample dset_target_imgs, with id=451
	Processing sample dset_target_imgs, with id=428
	Processing sample dset_target_imgs, with id=369
	Processing sample dset_target_imgs, with id=313
	Processing sample dset_target_imgs, with id=798
	Processing sample dset_target_imgs, with id=200
	Processing sample dset_target_imgs, with id=158
	Processing sample dse

	Processing sample dset_target_imgs, with id=526
	Processing sample dset_target_imgs, with id=377
	Processing sample dset_target_imgs, with id=834
	Processing sample dset_target_imgs, with id=145
	Processing sample dset_target_imgs, with id=227
	Processing sample dset_target_imgs, with id=637
	Processing sample dset_target_imgs, with id=362
	Processing sample dset_target_imgs, with id=569
	Processing sample dset_target_imgs, with id=486
	Processing sample dset_target_imgs, with id=676
	Processing sample dset_target_imgs, with id=752
	Processing sample dset_target_imgs, with id=879
	Processing sample dset_target_imgs, with id=31
	Processing sample dset_target_imgs, with id=510
	Processing sample dset_target_imgs, with id=774
	Processing sample dset_target_imgs, with id=68
	Processing sample dset_target_imgs, with id=212
	Processing sample dset_target_imgs, with id=786
	Processing sample dset_target_imgs, with id=609
	Processing sample dset_target_imgs, with id=291
	Processing sample dse

	Processing sample dset_target_imgs, with id=365
	Processing sample dset_target_imgs, with id=680
	Processing sample dset_target_imgs, with id=441
	Processing sample dset_target_imgs, with id=241
	Processing sample dset_target_imgs, with id=209
	Processing sample dset_target_imgs, with id=349
	Processing sample dset_target_imgs, with id=492
	Processing sample dset_target_imgs, with id=713
	Processing sample dset_target_imgs, with id=232
	Processing sample dset_target_imgs, with id=584
	Processing sample dset_target_imgs, with id=347
	Processing sample dset_target_imgs, with id=600
	Processing sample dset_target_imgs, with id=772
	Processing sample dset_target_imgs, with id=751
	Processing sample dset_target_imgs, with id=333
	Processing sample dset_target_imgs, with id=740
	Processing sample dset_target_imgs, with id=213
	Processing sample dset_target_imgs, with id=247
	Processing sample dset_target_imgs, with id=355
	Processing sample dset_target_imgs, with id=294
	Processing sample d

	Processing sample dset_target_imgs, with id=608
	Processing sample dset_target_imgs, with id=737
	Processing sample dset_target_imgs, with id=330
	Processing sample dset_target_imgs, with id=659
	Processing sample dset_target_imgs, with id=175
	Processing sample dset_target_imgs, with id=675
	Processing sample dset_target_imgs, with id=401
	Processing sample dset_target_imgs, with id=831
	Processing sample dset_target_imgs, with id=565
	Processing sample dset_target_imgs, with id=419
	Processing sample dset_target_imgs, with id=41
	Processing sample dset_target_imgs, with id=298
	Processing sample dset_target_imgs, with id=766
	Processing sample dset_target_imgs, with id=42
	Processing sample dset_target_imgs, with id=849
	Processing sample dset_target_imgs, with id=539
	Processing sample dset_target_imgs, with id=368
	Processing sample dset_target_imgs, with id=634
	Processing sample dset_target_imgs, with id=22
	Processing sample dset_target_imgs, with id=50
	Processing sample dset_

	Processing sample dset_target_imgs, with id=163
	Processing sample dset_target_imgs, with id=512
	Processing sample dset_target_imgs, with id=629
	Processing sample dset_target_imgs, with id=670
	Processing sample dset_target_imgs, with id=141
	Processing sample dset_target_imgs, with id=114
	Processing sample dset_target_imgs, with id=736
	Processing sample dset_target_imgs, with id=109
	Processing sample dset_target_imgs, with id=339
	Processing sample dset_target_imgs, with id=28
	Processing sample dset_target_imgs, with id=325
	Processing sample dset_target_imgs, with id=36
	Processing sample dset_target_imgs, with id=775
	Processing sample dset_target_imgs, with id=881
	Processing sample dset_target_imgs, with id=352
	Processing sample dset_target_imgs, with id=203
	Processing sample dset_target_imgs, with id=553
	Processing sample dset_target_imgs, with id=471
	Processing sample dset_target_imgs, with id=61
	Processing sample dset_target_imgs, with id=90
	Processing sample dset_

In [13]:
train_h5py.close()

In [14]:
val_h5py = h5py.File("data/val_all_classes.h5",mode='w')
dset_target_imgs = val_h5py.create_dataset("target_imgs", (0, PATCH_SIZE + 2*PAD_SIZE, PATCH_SIZE + 2*PAD_SIZE, 1),
                                             maxshape=(None, PATCH_SIZE + 2*PAD_SIZE, PATCH_SIZE + 2*PAD_SIZE, 1),
                                             dtype=np.uint8, shuffle=True, compression="gzip")

dset_imgs = val_h5py.create_dataset("imgs", (0, PATCH_SIZE, PATCH_SIZE, 3),
                                      maxshape=(None, PATCH_SIZE, PATCH_SIZE, 3),
                                      dtype=np.uint8, shuffle=True, compression="gzip")

dset_counts = val_h5py.create_dataset("counts", (0,), maxshape=(None,),
                                        dtype=np.uint8, shuffle=True, compression="gzip")

In [15]:
%%time

create_datasets(ID_VAL_IMGS, dset_target_imgs, dset_imgs, dset_counts)
print(dset_imgs.shape)
print(dset_target_imgs.shape)
print(dset_counts.shape)

val_h5py.close()

	Processing sample dset_target_imgs, with id=602
	Processing sample dset_target_imgs, with id=444
	Processing sample dset_target_imgs, with id=944
	Processing sample dset_target_imgs, with id=117
	Processing sample dset_target_imgs, with id=15
	Processing sample dset_target_imgs, with id=47
	Processing sample dset_target_imgs, with id=763
	Processing sample dset_target_imgs, with id=873
	Processing sample dset_target_imgs, with id=802
	Processing sample dset_target_imgs, with id=865
	Processing sample dset_target_imgs, with id=795
	Processing sample dset_target_imgs, with id=724
	Processing sample dset_target_imgs, with id=673
	Processing sample dset_target_imgs, with id=731
	Processing sample dset_target_imgs, with id=679
	Processing sample dset_target_imgs, with id=182
	Processing sample dset_target_imgs, with id=791
	Processing sample dset_target_imgs, with id=686
	Processing sample dset_target_imgs, with id=5
	Processing sample dset_target_imgs, with id=8
	Processing sample dset_ta